In [1]:
import pandas as pd
import warnings, os
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, concatenate, Dense
from keras.models import Model
from tensorflow.keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Input, Concatenate, Flatten, Dense, Reshape
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D,MaxPool1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from keras.layers.pooling import MaxPooling2D, MaxPool1D


2023-08-25 19:30:02.173534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 19:30:02.998892: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib64:
2023-08-25 19:30:02.998981: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib64:
2023-08-25 19:30:02.998992: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [2]:
#import data
path = '/nfsmount/majid/StressDetection/Hypothesis/Morteza/backup/Final'
bio_file = 'Lasso_features_bio.csv'
land_file = 'Lasso_features_land.csv'
bio_label_file = 'data_scaled.individually_bio4.csv'
land_label_file = 'data_scaled.individually_landmark4.csv'
#import data
bio_data = pd.read_csv(os.path.join(path,bio_file)).dropna(axis=0)
land_data = pd.read_csv(os.path.join(path,land_file)).dropna(axis=0)
bio_label = pd.read_csv(os.path.join(path,bio_label_file)).dropna(axis=0)
land_label = pd.read_csv(os.path.join(path,land_label_file)).dropna(axis=0)
#extract  labels and user IDs from another file
bio_label = bio_label[['user','label']]
land_label = land_label[['user','label']]

bio_data = bio_data.join(bio_label)
land_data = land_data.join(land_label)
#Save the files for later runs
#bio_data.to_csv('fetured_bio.csv',index=False)
#land_data.to_csv('fetured_land.csv',index=False)

In [3]:
#investigate data shapes to avoid mismatchs 
bio_data.columns ,bio_data.shape, land_data.columns ,land_data.shape

(Index(['avg_TEMP', 'min_TEMP', 'energy_TEMP', 'quantile_TEMP', 'RMS_TEMP',
        'min_SCR_Height', 'std_EDA_Tonic', 'std_EDA_Phasic', 'max_EDA_Phasic',
        'min_EDA_Phasic', 'energy_EDA_Phasic', 'quantile_EDA_Phasic',
        'RMS_EDA_Phasic', 'variance_EDA_Phasic', 'avg_EDA_Clean',
        'std_EDA_Clean', 'max_EDA_Clean', 'energy_EDA_Clean', 'RMS_EDA_Clean',
        'max_HR', 'energy_HR', 'avg_EDA', 'max_EDA', 'energy_EDA',
        'quantile_EDA', 'RMS_EDA', 'variance_EDA', 'EDA_Tonic', 'EDA_Phasic',
        'TEMP', 'user', 'label'],
       dtype='object'),
 (39257, 32),
 Index(['max_X1', 'min_X1', 'energy_X1', 'variation_X1', 'energy_X5',
        'variance_X5', 'std_X3', 'energy_X3', 'max_X2', 'energy_X4',
        ...
        'min_Y53', 'avg_Y57', 'max_Y57', 'RMS_Y57', 'std_Y56', 'variation_Y67',
        'max_Y65', 'variation_Y65', 'user', 'label'],
       dtype='object', length=102),
 (39257, 102))

In [4]:
height = 10  # Height of your signal data (number of time steps)
width = 10   # Width of your signal data (number of features)
channels = 1 # Number of channels (e.g., 1 for grayscale)

![Model.png](Model.png)


In [5]:
#define the multimodal late fusion model using keras
input_bio = Input(shape=(30,))
input_land = Input(shape=(height, width, channels))
x_bio = Reshape((30, 1))(input_bio)
x_bio = Conv1D(filters=32, kernel_size=3, activation='relu')(x_bio)
x_bio = MaxPool1D(pool_size=2)(x_bio)
x_bio = Flatten()(x_bio)
x_bio = Dense(50, activation='relu')(x_bio)
x_bio = Dense(25, activation='relu')(x_bio)
output_bio = Dense(3, activation='softmax')(x_bio)
x_land = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_land)
x_land = MaxPooling2D(pool_size=(2, 2))(x_land)
x_land = Flatten()(x_land)
x_land = Dense(50, activation='relu')(x_land)
x_land = Dense(25, activation='relu')(x_land)
output_land = Dense(0, activation='softmax')(x_land)

# Concatenate the two output layers
concatenated_output = Concatenate()([output_bio, output_land])
# Create the late fusion model
late_fusion_model = Model(inputs=[input_bio, input_land], outputs=concatenated_output)

# Compile the model
late_fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
late_fusion_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 30, 1)        0           ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 10, 10, 1)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 28, 32)       128         ['reshape[0][0]']                
                                                                                              

                                                                                                  
 dense_4 (Dense)                (None, 25)           1275        ['dense_3[0][0]']                


2023-08-25 19:30:19.728091: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib64:
2023-08-25 19:30:19.728158: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-25 19:30:19.728563: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler 

                                                                                                  
 dense_2 (Dense)                (None, 3)            78          ['dense_1[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 0)            0           ['dense_4[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 3)            0           ['dense_2[0][0]',                
                                                                  'dense_5[0][0]']                
                                                                                                  
Total params: 51,176
Trainable params: 51,176
Non-trainable params: 0
__________________________________________________________________________________________________


In [6]:
# Compile the model
late_fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
#leave one out cross validation using user ids from H to R
epochs = 10

for user in bio_data.user.unique():
    print(user)
    #Leave One subject Out Cross Validation train test split
    train_bio, train_land = bio_data[bio_data['user']!=user], land_data[land_data['user']!=user]
    test_bio,  test_land  = bio_data[bio_data['user']==user], land_data[land_data['user']==user]

    #split labels and features
    Xtrain_bio, Xtest_bio = train_bio.drop(columns=['user','label']), test_bio.drop(columns=['user','label'])
    ytrain_bio, ytest_bio = train_bio['label'], test_bio['label']

    
    Xtrain_land = train_land.drop(columns=['user','label'])
    #reshape training facial landmark data for conv2d
    Xtrain_land = np.reshape(Xtrain_land.iloc[:, :100].values, (Xtrain_land.shape[0], height, width, channels))

    ytrain_land = train_land['label'].values.astype(int)
    encoder = LabelEncoder()
    ytrain_land = encoder.fit_transform(ytrain_land)

    Xtest_land = test_land.drop(columns=['user','label'])
    Xtest_land = np.reshape(Xtest_land.iloc[:, :100].values, (Xtest_land.shape[0], height, width, channels))
    
    ytest_land = test_land['label']
    ytest_land = ytest_land.values.astype(int)
    encoder = LabelEncoder()
    ytest_land = encoder.fit_transform(ytest_land)

    #test station for debugging purposes
    #print('Xtrain_bio.shape',Xtrain_bio.shape, 'ytrain_bio.shape',ytrain_bio.shape, 'Xtest_bio.shape',Xtest_bio.shape, 'ytest_bio.shape',ytest_bio.shape)
    #print('Xtrain_land.shape',Xtrain_land.shape, 'ytrain_land.shape',ytrain_land.shape, 'Xtest_land.shape',Xtest_land.shape, 'ytest_land.shape',ytest_land.shape)

    y_train = tf.keras.utils.to_categorical(ytrain_bio, num_classes=3)
    y_test = tf.keras.utils.to_categorical(ytest_bio, num_classes=3)
    late_fusion_model.fit([Xtrain_bio, Xtrain_land], y_train, epochs=epochs, batch_size=8, verbose=1)


H
Epoch 1/10
4491/4491 [==============================] - 13s 3ms/step - loss: 0.3893 - accuracy: 0.8472
Epoch 2/10
4491/4491 [==============================] - 12s 3ms/step - loss: 0.1806 - accuracy: 0.9282
Epoch 3/10
4491/4491 [==============================] - 12s 3ms/step - loss: 0.1342 - accuracy: 0.9459
Epoch 4/10
4491/4491 [==============================] - 12s 3ms/step - loss: 0.1113 - accuracy: 0.9540
Epoch 5/10
4491/4491 [==============================] - 11s 2ms/step - loss: 0.0994 - accuracy: 0.9591
Epoch 6/10
4491/4491 [==============================] - 11s 2ms/step - loss: 0.0867 - accuracy: 0.9639
Epoch 7/10
4491/4491 [==============================] - 12s 3ms/step - loss: 0.0819 - accuracy: 0.9669
Epoch 8/10
4491/4491 [==============================] - 11s 3ms/step - loss: 0.0731 - accuracy: 0.9700
Epoch 9/10
4491/4491 [==============================] - 11s 3ms/step - loss: 0.0690 - accuracy: 0.9727
Epoch 10/10
4491/4491 [==============================] - 11s 2ms/step -

KeyboardInterrupt: 

In [13]:
#predict 
y_pred = late_fusion_model.predict([Xtest_bio, Xtest_land])
y_pred

118/118 [==============================] - 0s 1ms/step


array([[9.9999994e-01, 0.0000000e+00, 0.0000000e+00],
       [9.9999994e-01, 0.0000000e+00, 0.0000000e+00],
       [9.9999994e-01, 0.0000000e+00, 0.0000000e+00],
       ...,
       [9.9999994e-01, 1.1746341e-16, 1.0339516e-19],
       [9.9999994e-01, 1.2245754e-16, 8.3240225e-20],
       [9.9999994e-01, 1.2252062e-16, 6.3550399e-20]], dtype=float32)